# Preprocessamento e criação do modelo

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
import tensorflow as tf
import joblib
from fuzzywuzzy import process 
from datetime import datetime
import random as python_random
from utils import *


In [33]:

seed = 41
np.random.seed(seed)
python_random.seed(seed)
tf.random.set_seed(seed)

In [34]:
df = pd.read_csv('./data/bronze/data.csv')

In [35]:
# Conversão de tipos
df['idade'] = df['idade'].astype(int)
df['valorsolicitado'] = df['valorsolicitado'].astype(float)
df['valortotalbem'] = df['valortotalbem'].astype(float)

## Tratamento de Nulos

In [36]:
fill_na(df)

## Tratamento de erros de digitação

In [37]:
profissoes_validas = ['Advogado','Arquiteto','Cientista de Dados',
                      'Contador','Dentista','Empresário','Engenheiro',
                      'Médico','Programador']
fix_categoric_errors(df,'profissao', profissoes_validas)

## Tratamento de Outliers

In [38]:
df = fix_outliers(df, 'idade', 18, 100)
df = fix_outliers(df, 'idade', 0, 110)

## Engenharia de features

In [39]:
df['proporcaosolicitadototal'] = (df['valorsolicitado'] / df['valortotalbem']).astype(float)

## Divisão de dados para o modelo

In [40]:
X = df.drop('classe', axis=1)
y = df['classe']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

# Normalização

X_test = save_scalers(X_test, ['tempoprofissao','renda','idade','dependentes','valorsolicitado',
                               'valortotalbem','proporcaosolicitadototal'])
X_train = save_scalers(X_train, ['tempoprofissao','renda','idade','dependentes','valorsolicitado',
                               'valortotalbem','proporcaosolicitadototal'])

# Label Encoding
mapeamento = {'ruim':0, 'bom':1}
y_train = np.array([mapeamento[classe] for classe in y_train])
y_test = np.array([mapeamento[classe] for classe in y_test])

X_train = save_encoders(X_train, ['profissao','tiporesidencia','escolaridade',
                                  'score','estadocivil','produto'])
X_test = save_encoders(X_train, ['profissao','tiporesidencia','escolaridade',
                                  'score','estadocivil','produto'])

# Feature Selection
rf = RandomForestClassifier()
selector = RFE(rf, n_features_to_select=10, step=1)
selector.fit(X_train, y_train)

# Transformação dos dados

X_train = selector.transform(X_train)
X_test = selector.transform(X_test)

# Salvar seletor
joblib.dump(selector, './models/selector.joblib')


['./models/selector.joblib']